<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-07-19 15:03:20


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
def print_distribution():

  # star stocks
  print('Star Stocks:')
  print(df_mypf['StarStock'].value_counts())

  # v40/v40n stocks
  print('V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())

  # stocks in portfolio
  print('Stocks in Portfolio:')
  print(df_mypf['InPortfolio'].value_counts())

  # star stocks in portfolio
  print('Star Stocks in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())

  # criteria of portfolio stocks
  print('Criteria in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())

  # strategies in portfolio stocks
  print('Strategy in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())

  # conviction distribution
  print('Conviction in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Star Stocks:
0.0    94
1.0    89
Name: StarStock, dtype: int64
V40/V40N Stocks:
V40N    39
V40     38
VR      21
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
Latest Qtr:
0    170
1     15
Name: LatestQtr, dtype: int64
Star + V40/V40N Stocks:
1.0    41
0.0    36
Name: StarStock, dtype: int64
Stocks in Portfolio:
0    136
1     49
Name: InPortfolio, dtype: int64
Star Stocks in Portfolio:
1.0    33
0.0    16
Name: StarStock, dtype: int64
Criteria in Portfolio:
VR      21
V40N    11
V40     10
AR       4
V200     3
Name: Criteria, dtype: int64
Strategy in Portfolio:
ATH     34
2T3Y    14
Name: Strategy, dtype: int64
Conviction in Portfolio:
M    25
L    13
H    11
Name: Conviction, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [9]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
    tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:

# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-06-20 00:00:00+05:30,18817.0,^NSEI,5.05,0.29,0.75,65.0,19814.0,7610.0,8.17,5.0
2023-06-27 00:00:00+05:30,18817.0,^NSEI,4.88,0.33,0.56,61.0,19814.0,7610.0,8.17,5.0
2023-07-05 00:00:00+05:30,19398.0,^NSEI,7.91,0.36,0.86,76.0,19814.0,7610.0,3.41,2.0
2023-07-12 00:00:00+05:30,19384.0,^NSEI,7.59,0.42,0.78,68.0,19814.0,7610.0,3.52,2.0
2023-07-19 00:00:00+05:30,19814.0,^NSEI,9.62,0.54,0.78,78.0,19814.0,7610.0,0.00,0.0


In [11]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

df_mypf['Stock'] = df_mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
df_prec_dev["Target"] = df_prec_dev["Target"].fillna(df_prec_dev["Max"])
df_prec_dev['Upside%Target'] = round((df_prec_dev['Target']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [13]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%Target',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'RSI', 'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 49


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
BANDHANBNK.NS,214.0,-9.58,-1.27,-0.82,89.46,238.0,244.52,NaN,17.00,31.0,M,NaN,1,0.0,VR,2T3Y
CLEAN.NS,1316.0,-9.97,-1.85,-1.21,94.54,101.0,1365.11,NaN,47.40,35.0,L,NaN,0,1.0,VR,2T3Y
PAGEIND.NS,36459.0,-13.10,-1.58,-1.06,45.63,47.0,38548.77,97.7,71.20,36.0,H,27.0,0,1.0,V40,ATH
VALIANTORG.NS,507.0,-8.15,-2.54,-0.19,91.37,265.0,532.61,24.0,16.20,38.0,L,33.0,0,0.0,VR,2T3Y
RAJESHEXPO.NS,520.0,-21.87,-0.45,-0.89,84.28,82.0,578.92,NaN,10.70,39.0,M,NaN,0,1.0,V40N,ATH
JUBLFOOD.NS,476.0,-4.77,-1.40,0.21,61.15,89.0,471.56,90.5,88.90,43.0,M,2.0,0,1.0,AR,ATH
EICHERMOT.NS,3330.0,-0.77,-0.54,-0.11,20.02,16.0,3260.20,44.7,31.20,43.0,H,30.0,0,1.0,V40N,ATH
AMBUJACEM.NS,419.0,-7.20,-1.11,0.70,35.37,39.0,424.02,23.9,29.70,43.0,L,-24.0,0,0.0,VR,ATH
AMARAJABAT.NS,649.0,8.90,1.51,1.25,54.85,52.0,625.25,NaN,15.20,44.0,M,NaN,0,1.0,V200,ATH


In [14]:
# top from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI', ascending=True).head(10)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
BANDHANBNK.NS,214.0,-9.58,-1.27,-0.82,89.46,238.0,244.52,NaN,17.0,31.0,M,NaN,1,0.0,VR,2T3Y
CLEAN.NS,1316.0,-9.97,-1.85,-1.21,94.54,101.0,1365.11,NaN,47.4,35.0,L,NaN,0,1.0,VR,2T3Y
PAGEIND.NS,36459.0,-13.10,-1.58,-1.06,45.63,47.0,38548.77,97.7,71.2,36.0,H,27.0,0,1.0,V40,ATH
VALIANTORG.NS,507.0,-8.15,-2.54,-0.19,91.37,265.0,532.61,24.0,16.2,38.0,L,33.0,0,0.0,VR,2T3Y
RAJESHEXPO.NS,520.0,-21.87,-0.45,-0.89,84.28,82.0,578.92,NaN,10.7,39.0,M,NaN,0,1.0,V40N,ATH
AMBUJACEM.NS,419.0,-7.20,-1.11,0.70,35.37,39.0,424.02,23.9,29.7,43.0,L,-24.0,0,0.0,VR,ATH
EICHERMOT.NS,3330.0,-0.77,-0.54,-0.11,20.02,16.0,3260.20,44.7,31.2,43.0,H,30.0,0,1.0,V40N,ATH
JUBLFOOD.NS,476.0,-4.77,-1.40,0.21,61.15,89.0,471.56,90.5,88.9,43.0,M,2.0,0,1.0,AR,ATH
DMART.NS,3704.0,-1.72,-0.80,0.52,38.66,44.0,3659.46,160.1,101.0,44.0,M,37.0,1,1.0,AR,ATH


In [15]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
PIDILITIND.NS,2679.0,6.05,-0.40,0.69,11.13,8.0,2504.06,94.6,107.0,60.0,H,-13.0,0,1.0,V40,ATH
TCS.NS,3472.0,7.52,0.62,0.67,16.61,11.0,3231.00,28.1,29.0,67.0,H,-3.0,1,1.0,V40,ATH
BAJAJFINSV.NS,1642.0,10.57,-0.47,1.67,17.58,16.0,1454.93,39.2,40.7,74.0,H,-4.0,0,1.0,V40,ATH
EICHERMOT.NS,3330.0,-0.77,-0.54,-0.11,20.02,16.0,3260.20,44.7,31.2,43.0,H,30.0,0,1.0,V40N,ATH
ICICIGI.NS,1366.0,17.42,0.69,2.36,27.44,18.0,1177.58,46.2,1000.0,69.0,H,-2065.0,1,1.0,V40,ATH
ITBEES.NS,33.0,9.67,0.00,0.00,29.17,21.0,31.49,NaN,1000.0,78.0,L,NaN,0,1.0,AR,ATH
HDFCBANK.NS,1685.0,6.06,0.57,0.00,4.38,22.0,1683.55,22.0,26.0,60.0,H,-18.0,1,1.0,V40,NaN


In [16]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_prec_dev[~df_prec_dev.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]

In [17]:
# star + v40/v40n stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=False)

qualified stocks: 4


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
ASTRAZEN.NS,3809.0,14.04,0.81,1.68,71.0,4631.0,1311.0,24.76,22.0,4631.0,M,0,1.0,V40N,NaN,22.0
BERGEPAINT.NS,682.0,12.59,0.17,0.76,61.0,858.0,260.0,29.43,26.0,858.0,M,0,1.0,V40,NaN,26.0
DIXON.NS,4330.0,18.31,-0.22,3.61,57.0,5750.0,374.0,26.41,33.0,5750.0,M,0,1.0,V40N,NaN,33.0
ANGELONE.NS,1505.0,11.26,0.81,2.89,37.0,1889.0,207.0,22.83,26.0,1889.0,M,1,1.0,V40N,NaN,26.0


In [18]:
# star + v40/v40n stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
BAJFINANCE.NS,7572.0,15.77,0.12,1.67,63.0,7860.0,1804.0,4.76,4.0,7860.0,H,0,1.0,V40,ATH,4.0
MCDOWELL-N.NS,969.0,15.54,0.48,1.93,72.0,1004.0,448.0,6.29,4.0,1004.0,L,0,1.0,V40N,NaN,4.0
AKZOINDIA.NS,2683.0,17.17,1.22,1.33,67.0,2764.0,1360.0,5.77,3.0,2764.0,M,0,1.0,V40,NaN,3.0
HINDUNILVR.NS,2668.0,3.96,0.23,0.75,50.0,2757.0,1394.0,6.53,3.0,2757.0,H,0,1.0,V40,NaN,3.0
ABBOTINDIA.NS,23375.0,10.89,1.33,0.39,58.0,23826.0,6634.0,2.62,2.0,23826.0,M,0,1.0,V40,NaN,2.0
BAJAJ-AUTO.NS,4823.0,27.36,1.80,1.43,64.0,4918.0,1695.0,2.95,2.0,4918.0,L,0,1.0,V40,NaN,2.0
ASIANPAINT.NS,3508.0,15.69,0.10,1.54,76.0,3542.0,1098.0,1.39,1.0,3542.0,H,0,1.0,V40,ATH,1.0
COLPAL.NS,1829.0,17.61,0.77,1.62,76.0,1839.0,931.0,1.10,1.0,1839.0,H,0,1.0,V40,NaN,1.0
EQUITASBNK.NS,96.0,45.68,3.03,3.45,69.0,96.0,33.0,0.00,0.0,96.0,M,0,1.0,V40N,NaN,0.0


In [19]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
UPL.NS,640.0,-10.65,-0.42,-1.33,35.0,832.0,245.0,32.71,30.0,832.0,L,0,1.0,NaN,NaN,30.0
GODREJIND.NS,481.0,7.99,0.22,0.83,44.0,644.0,235.0,39.85,34.0,644.0,L,0,1.0,NaN,NaN,34.0
TATAPOWER.NS,220.0,4.52,-0.48,0.93,47.0,285.0,26.0,25.10,30.0,285.0,M,0,1.0,NaN,NaN,30.0
ADANITRANS.NS,784.0,-55.80,-9.08,-2.85,50.0,4105.0,148.0,83.93,424.0,4105.0,L,0,1.0,NaN,NaN,424.0
SBICARD.NS,867.0,7.30,-0.62,0.57,55.0,1133.0,496.0,41.76,31.0,1133.0,M,0,1.0,NaN,NaN,31.0
ADANIENT.NS,2426.0,-11.31,-2.34,2.14,57.0,4161.0,103.0,42.76,72.0,4161.0,M,0,1.0,NaN,NaN,72.0
ADANIGREEN.NS,980.0,-28.45,-4.97,0.21,58.0,2970.0,30.0,67.69,203.0,2970.0,L,0,1.0,NaN,NaN,203.0


In [20]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
EQUITASBNK.NS,96.0,45.68,3.03,3.45,69.0,96.0,33.0,0.00,0.0,96.0,M,0,1.0,V40N,NaN,0.0
ICICIBANK.NS,973.0,8.22,0.33,0.42,69.0,973.0,258.0,0.00,0.0,973.0,H,0,1.0,V40,NaN,0.0
ITC.NS,477.0,26.51,1.86,1.12,71.0,477.0,124.0,0.00,0.0,477.0,H,0,1.0,V40,NaN,0.0
POLYCAB.NS,4300.0,44.25,1.98,1.94,86.0,4300.0,523.0,0.00,0.0,4300.0,M,1,1.0,V40N,NaN,0.0
RELIANCE.NS,2823.0,14.14,0.44,1.30,78.0,2823.0,867.0,0.00,0.0,2823.0,H,0,1.0,V40,ATH,0.0
ASIANPAINT.NS,3508.0,15.69,0.10,1.54,76.0,3542.0,1098.0,1.39,1.0,3542.0,H,0,1.0,V40,ATH,1.0
COLPAL.NS,1829.0,17.61,0.77,1.62,76.0,1839.0,931.0,1.10,1.0,1839.0,H,0,1.0,V40,NaN,1.0
ABBOTINDIA.NS,23375.0,10.89,1.33,0.39,58.0,23826.0,6634.0,2.62,2.0,23826.0,M,0,1.0,V40,NaN,2.0
BAJAJ-AUTO.NS,4823.0,27.36,1.80,1.43,64.0,4918.0,1695.0,2.95,2.0,4918.0,L,0,1.0,V40,NaN,2.0


In [21]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
NESTLEIND.NS,22940.0,13.41,1.09,0.51,56.0,23138.0,8621.0,1.36,1.0,23138.0,H,0,0.0,V40,NaN,1.0
PGHH.NS,16009.0,15.01,0.24,1.06,82.0,16009.0,8145.0,0.00,0.0,16009.0,L,0,0.0,V40,NaN,0.0


In [22]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,


In [23]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
SYMPHONY.NS,871.0,-7.53,0.00,-0.90,36.0,1470.0,707.0,78.51,69.0,1470.0,M,0,0.0,V40N,NaN,69.0
JCHAC.NS,1033.0,-9.36,-1.93,-0.85,44.0,3219.0,1017.0,99.27,212.0,3219.0,M,0,0.0,V40N,NaN,212.0
WHIRLPOOL.NS,1435.0,-0.02,-0.91,0.77,44.0,2667.0,1254.0,87.19,86.0,2667.0,M,0,0.0,V40,NaN,86.0
SIS.NS,425.0,9.51,-0.26,1.23,56.0,605.0,321.0,63.38,42.0,605.0,M,0,0.0,V40N,NaN,42.0
PFIZER.NS,3865.0,-4.03,-0.50,0.10,60.0,6009.0,2301.0,57.82,55.0,6009.0,L,0,0.0,V40,NaN,55.0
MOTILALOFS.NS,745.0,12.51,-0.15,2.22,62.0,1091.0,439.0,53.07,46.0,1091.0,L,0,0.0,V40N,NaN,46.0
LALPATHLAB.NS,2344.0,8.37,-0.09,1.94,64.0,4186.0,807.0,54.51,79.0,4186.0,M,0,0.0,V40N,NaN,79.0
SFL.NS,1239.0,3.18,-1.33,1.08,65.0,1972.0,566.0,52.13,59.0,1972.0,M,0,0.0,V40N,NaN,59.0
TTKPRESTIG.NS,786.0,-0.69,-1.52,0.82,69.0,1165.0,391.0,48.97,48.0,1165.0,M,0,0.0,V40N,NaN,48.0


In [24]:
# prospects near 52W low
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 4


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
SYMPHONY.NS,871.0,-7.53,0.00,-0.90,36.0,1470.0,707.0,78.51,69.0,1470.0,M,0,0.0,V40N,NaN,69.0
JCHAC.NS,1033.0,-9.36,-1.93,-0.85,44.0,3219.0,1017.0,99.27,212.0,3219.0,M,0,0.0,V40N,NaN,212.0
WHIRLPOOL.NS,1435.0,-0.02,-0.91,0.77,44.0,2667.0,1254.0,87.19,86.0,2667.0,M,0,0.0,V40,NaN,86.0
SIS.NS,425.0,9.51,-0.26,1.23,56.0,605.0,321.0,63.38,42.0,605.0,M,0,0.0,V40N,NaN,42.0


In [25]:
# prospects near 52W high
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 28


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
FINCABLES.NS,972.0,42.96,2.79,-0.36,73.0,972.0,165.0,0.00,0.0,972.0,M,0,0.0,V40N,NaN,0.0
RELIANCE.NS,2823.0,14.14,0.44,1.30,78.0,2823.0,867.0,0.00,0.0,2823.0,H,0,1.0,V40,ATH,0.0
EQUITASBNK.NS,96.0,45.68,3.03,3.45,69.0,96.0,33.0,0.00,0.0,96.0,M,0,1.0,V40N,NaN,0.0
PGHH.NS,16009.0,15.01,0.24,1.06,82.0,16009.0,8145.0,0.00,0.0,16009.0,L,0,0.0,V40,NaN,0.0
POLYCAB.NS,4300.0,44.25,1.98,1.94,86.0,4300.0,523.0,0.00,0.0,4300.0,M,1,1.0,V40N,NaN,0.0
ITC.NS,477.0,26.51,1.86,1.12,71.0,477.0,124.0,0.00,0.0,477.0,H,0,1.0,V40,NaN,0.0
ICICIBANK.NS,973.0,8.22,0.33,0.42,69.0,973.0,258.0,0.00,0.0,973.0,H,0,1.0,V40,NaN,0.0
COLPAL.NS,1829.0,17.61,0.77,1.62,76.0,1839.0,931.0,1.10,1.0,1839.0,H,0,1.0,V40,NaN,1.0
RADICO.NS,1378.0,22.96,1.25,2.22,67.0,1393.0,241.0,1.30,1.0,1393.0,M,0,0.0,V40N,NaN,1.0


In [26]:
# prospects in bull run
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 27


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
EQUITASBNK.NS,96.0,45.68,3.03,3.45,69.0,96.0,33.0,0.00,0.0,96.0,M,0,1.0,V40N,NaN,0.0
ICICIPRULI.NS,559.0,19.02,0.21,2.93,51.0,716.0,237.0,32.78,28.0,716.0,M,1,0.0,V40,NaN,28.0
ANGELONE.NS,1505.0,11.26,0.81,2.89,37.0,1889.0,207.0,22.83,26.0,1889.0,M,1,1.0,V40N,NaN,26.0
NAM-INDIA.NS,298.0,26.66,0.85,2.85,80.0,413.0,111.0,38.08,39.0,413.0,L,0,0.0,V40,NaN,39.0
ISEC.NS,618.0,23.07,0.80,2.77,73.0,829.0,164.0,31.73,34.0,829.0,M,0,0.0,V40N,NaN,34.0
RADICO.NS,1378.0,22.96,1.25,2.22,67.0,1393.0,241.0,1.30,1.0,1393.0,M,0,0.0,V40N,NaN,1.0
SANOFI.NS,7013.0,22.89,1.12,2.16,63.0,8070.0,4096.0,26.60,15.0,8070.0,M,0,0.0,V40,NaN,15.0
HDFCLIFE.NS,658.0,18.83,0.90,2.14,57.0,754.0,339.0,23.13,15.0,754.0,H,0,0.0,V40,NaN,15.0
3MINDIA.NS,28048.0,18.71,1.16,2.10,58.0,29373.0,15332.0,9.44,5.0,29373.0,M,0,1.0,V40N,NaN,5.0


In [27]:
# prospects about to turn around
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
JCHAC.NS,1033.0,-9.36,-1.93,-0.85,44.0,3219.0,1017.0,99.27,212.0,3219.0,M,0,0.0,V40N,NaN,212.0
WHIRLPOOL.NS,1435.0,-0.02,-0.91,0.77,44.0,2667.0,1254.0,87.19,86.0,2667.0,M,0,0.0,V40,NaN,86.0
TATAELXSI.NS,7549.0,10.30,-0.89,1.29,51.0,10602.0,525.0,30.30,40.0,10602.0,M,0,0.0,V40N,NaN,40.0
VINATIORGA.NS,1842.0,-4.50,-1.04,-0.70,55.0,2342.0,453.0,26.47,27.0,2342.0,M,0,0.0,V40N,NaN,27.0
SIS.NS,425.0,9.51,-0.26,1.23,56.0,605.0,321.0,63.38,42.0,605.0,M,0,0.0,V40N,NaN,42.0
DIXON.NS,4330.0,18.31,-0.22,3.61,57.0,5750.0,374.0,26.41,33.0,5750.0,M,0,1.0,V40N,NaN,33.0
PFIZER.NS,3865.0,-4.03,-0.50,0.10,60.0,6009.0,2301.0,57.82,55.0,6009.0,L,0,0.0,V40,NaN,55.0
KANSAINER.NS,316.0,11.13,-0.35,1.71,61.0,436.0,192.0,49.18,38.0,436.0,M,0,0.0,V40N,NaN,38.0
MOTILALOFS.NS,745.0,12.51,-0.15,2.22,62.0,1091.0,439.0,53.07,46.0,1091.0,L,0,0.0,V40N,NaN,46.0


In [28]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
SYMPHONY.NS,871.0,-7.53,0.00,-0.90,36.0,1470.0,707.0,78.51,69.0,1470.0,M,0,0.0,V40N,NaN,69.0
JCHAC.NS,1033.0,-9.36,-1.93,-0.85,44.0,3219.0,1017.0,99.27,212.0,3219.0,M,0,0.0,V40N,NaN,212.0
WHIRLPOOL.NS,1435.0,-0.02,-0.91,0.77,44.0,2667.0,1254.0,87.19,86.0,2667.0,M,0,0.0,V40,NaN,86.0
SIS.NS,425.0,9.51,-0.26,1.23,56.0,605.0,321.0,63.38,42.0,605.0,M,0,0.0,V40N,NaN,42.0
PFIZER.NS,3865.0,-4.03,-0.50,0.10,60.0,6009.0,2301.0,57.82,55.0,6009.0,L,0,0.0,V40,NaN,55.0
MOTILALOFS.NS,745.0,12.51,-0.15,2.22,62.0,1091.0,439.0,53.07,46.0,1091.0,L,0,0.0,V40N,NaN,46.0
LALPATHLAB.NS,2344.0,8.37,-0.09,1.94,64.0,4186.0,807.0,54.51,79.0,4186.0,M,0,0.0,V40N,NaN,79.0
SFL.NS,1239.0,3.18,-1.33,1.08,65.0,1972.0,566.0,52.13,59.0,1972.0,M,0,0.0,V40N,NaN,59.0
TTKPRESTIG.NS,786.0,-0.69,-1.52,0.82,69.0,1165.0,391.0,48.97,48.0,1165.0,M,0,0.0,V40N,NaN,48.0


In [29]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Target,Conviction,LatestQtr,StarStock,Criteria,Strategy,Upside%Target
Stock,,,,,,,,,,,,,,,,
PVRINOX.NS,1432.0,-10.38,-0.75,-0.56,54.0,1917.0,1348.0,85.24,34.0,1917.0,L,0,0.0,NaN,NaN,34.0


In [30]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [31]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-06-19 00:00:00+05:30,676.0,BERGEPAINT.NS,11.85,-0.17,1.44,75.0,858.0,260.0,30.43,27.0
2023-06-26 00:00:00+05:30,677.0,BERGEPAINT.NS,12.09,0.00,1.42,66.0,858.0,260.0,30.27,27.0
2023-07-04 00:00:00+05:30,670.0,BERGEPAINT.NS,10.82,0.17,1.55,58.0,858.0,260.0,31.44,28.0
2023-07-11 00:00:00+05:30,670.0,BERGEPAINT.NS,10.77,0.17,1.08,54.0,858.0,260.0,31.44,28.0
2023-07-19 00:00:00+05:30,682.0,BERGEPAINT.NS,12.52,0.17,0.76,61.0,858.0,260.0,29.43,26.0
